<a href="https://colab.research.google.com/github/rjshrd/SemanticSearch/blob/main/semantic_search_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



This script provides a basic implementation where:

- I created a PostgreSQL table with an embedding column for semantic search.
- generated dummy data for employees including their skills, resume text, and project history.
- Used `SentenceTransformer` for embedding generation since LLaMA isn't publicly available for direct use in this context.
- The search function uses cosine similarity to find matches based on the query.

**Note**: This is a very "hello world". we need more sophisticated data handling, better integration with actual resume parsing tools, more robust database interactions, and  using more advanced models or tools like ElasticSearch for full-text search capabilities. You'd also want to add error handling, logging, and possibly a UI if you're looking to make it more user-friendly.

1. **Install Necessary Libraries**:
   `psycopg2` for PostgreSQL,
   `pandas` for data manipulation,
   `sentence_transformers` for generating embeddings,
   `scikit-learn` for similarity computation.

In [1]:
!pip install pandas sentence-transformers scikit-learn

2. **Connect to PostgreSQL**:

In [2]:
# Step 1: Install PostgreSQL and pgvector
!apt-get update -y
!apt-get install postgresql postgresql-contrib -y
!apt-get install postgresql-server-dev-all -y  # Required for pgvector extension
!git clone https://github.com/pgvector/pgvector.git
!cd pgvector && make && make install

# Step 2: Start PostgreSQL Service
!service postgresql start

# Step 3: Initialize the database and enable pgvector
!sudo -u postgres psql -c "CREATE DATABASE colab_db;"
!sudo -u postgres psql -d colab_db -c "CREATE EXTENSION vector;"

# Step 4: Create user and grant privileges
!sudo -u postgres psql -c "CREATE USER colab_user WITH PASSWORD 'password';"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE colab_db TO colab_user;"

# Step 5: Connect and create the table with vector data type
import psycopg2

try:
    conn = psycopg2.connect(
        dbname="colab_db",  # The database we created
        user="colab_user",  # The user we created
        password="password",  # The password we set
        host="localhost",  # Localhost because it's running in Colab
        port="5432"  # Default PostgreSQL port
    )
    print("Connected to the database successfully!")
    # cur = conn.cursor()
    # Close the cursor and connection
    # cur.close()
    # conn.close()

except Exception as e:
    print("Error:", e)


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,192 kB]
Get:13 https://r2u.stat.illinois

In [3]:
# Create a cursor
cur = conn.cursor()

3. **Create a Simple Database Schema**:

In [4]:
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
cur.execute("""
   CREATE TABLE IF NOT EXISTS employees (
       id SERIAL PRIMARY KEY,
       name TEXT,
       skills TEXT,
       resume TEXT,
       project_history TEXT,
       embedding vector(384)
   );
   """)

4. **Generate dummy Data**:

In [5]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the model for generating embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create some dummy data
employees = pd.DataFrame({
       'name': ['A1', 'A2', 'A3'],
       'skills': ['Python, Machine Learning', 'JavaScript, React', 'SQL, Data Analysis'],
       'resume': ['A1 is a data scientist...', 'A2 specializes in frontend...', 'A3 has worked with big data...'],
       'project_history': ['Project A', 'Project B', 'Project C']
   })



/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Generate embeddings
skills_embeddings = model.encode(employees['skills'].tolist())
resume_embeddings = model.encode(employees['resume'].tolist())

# For simplicity, we'll combine skills and resume embeddings
combined_embeddings = [sum(x) for x in zip(skills_embeddings, resume_embeddings)]

# Insert into database
for i, row in employees.iterrows():
    cur.execute("INSERT INTO employees (name, skills, resume, project_history, embedding) VALUES (%s, %s, %s, %s, %s)",
                   (row['name'], row['skills'], row['resume'], row['project_history'], combined_embeddings[i].tolist()))
conn.commit()

**Semantic Search**

In [10]:
def semantic_search(query):
    try:
        # Generate the query embedding
        query_embedding = model.encode([query])[0]

        # Perform the SQL query
        cur.execute("""
            SELECT id, name, skills,
                   1 - (embedding <=> %s::vector) AS similarity  -- Compute similarity from distance
            FROM employees
            ORDER BY similarity DESC
            LIMIT 5;
        """, (query_embedding.tolist(),))

        results = cur.fetchall()

        # Print the results
        for row in results:
            print(f"ID: {row[0]}, Name: {row[1]}, Skills: {row[2]}, Similarity: {row[3]}")

    except Exception as e:
        print("Error during semantic search:", e)

In [11]:
# Search query
semantic_search("Looking for someone with skills in data analysis and machine learning")

ID: 1, Name: A1, Skills: Python, Machine Learning, Similarity: 0.49688906590085546
ID: 3, Name: A3, Skills: SQL, Data Analysis, Similarity: 0.38113304194684194
ID: 2, Name: A2, Skills: JavaScript, React, Similarity: 0.1309472557864806


In [12]:
#Close the cursor and connection
cur.close()
conn.close()